# Algoritmos de otimização - Calendário de voos

## Representação do problema

In [35]:
#para montar o Drive do Google nos "contents"
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**A representação dos voos será da seguinte forma:**

Vamos definir uma estutura para representar os voos em um MAPA, assim:

voos(k,v), onde:

k = origem,destino

v = hora de saída, hora de chegada, preço

EXEMPLO:

```
voos = {{'BRU', 'FCO'}: ['15:35', '18:40', '382']};

print(voos['BRU', 'FCO]) #IMPRIME TODAS AS INFOS DO VOO 
print(voos[('BRU', 'FCO')][0], #IMPRIME O HORÁRIO DE SAÍDA
voos[('BRU', 'FCO')][1], #IMPRIME O HORÁRIO DE CHEGADA
voos[('BRU', 'FCO')][2]) #IMPRIME O PREÇO
```


In [77]:
#cria o mapa para os voos:
voos = {}

for linha in open('/content/drive/MyDrive/Colab Notebooks/Aula 2 - Algoritmos de Otimização/voos.txt'):
  #só pra testar se o arquivo foi aberto...:
  #print(linha)
  #print(linha.split(',')) #split separa cada item da linha em uma posição em um vetor
  
  #criamos 5 variáveis para receber as infromações necessárias de cada voo
  origem, destino, saida, chegada, preco = linha.split(',')

  #gera uma chave (k) para um MAP (no caso, do voo):
  voos.setdefault((origem, destino), [])

  #acrescenta as informações de voo (v) no (k):
  voos[(origem, destino)].append((saida, chegada, int(preco)))

In [78]:
voos

{('BRU', 'FCO'): [('6:12', '10:22', 230),
  ('7:53', '11:37', 433),
  ('9:08', '12:12', 364),
  ('10:30', '14:57', 290),
  ('12:19', '15:25', 342),
  ('13:54', '18:02', 294),
  ('15:44', '18:55', 382),
  ('16:52', '20:48', 448),
  ('18:26', '21:29', 464),
  ('20:07', '23:27', 473)],
 ('CDG', 'FCO'): [('6:25', '9:30', 335),
  ('7:34', '9:40', 324),
  ('9:15', '12:29', 225),
  ('11:28', '14:40', 248),
  ('12:05', '15:30', 330),
  ('14:01', '17:24', 338),
  ('15:34', '18:11', 326),
  ('17:07', '20:04', 291),
  ('18:23', '21:35', 134),
  ('19:53', '22:21', 173)],
 ('DUB', 'FCO'): [('6:17', '8:26', 89),
  ('8:04', '10:11', 95),
  ('9:45', '11:50', 172),
  ('11:16', '13:29', 83),
  ('12:34', '15:02', 109),
  ('13:40', '15:37', 138),
  ('15:27', '17:18', 151),
  ('17:11', '18:30', 108),
  ('18:34', '19:36', 136),
  ('20:17', '22:22', 102)],
 ('FCO', 'BRU'): [('6:09', '9:49', 414),
  ('7:57', '11:15', 347),
  ('9:49', '13:51', 229),
  ('10:51', '14:16', 256),
  ('12:20', '16:34', 500),
  ('14:

In [38]:
#voos específicos de Lisboa até Roma (exemplo):
voos['LIS', 'FCO']

[('6:11', '8:31', 249),
 ('7:39', '10:24', 219),
 ('9:15', '12:03', 99),
 ('11:08', '13:07', 175),
 ('12:18', '14:56', 172),
 ('13:37', '15:08', 250),
 ('15:03', '16:42', 135),
 ('16:51', '19:09', 147),
 ('18:12', '20:17', 242),
 ('20:05', '22:06', 261)]

In [80]:
#representação dos viajantes (pessoas):

pessoas = [('Viajante 1', 'LIS'),  # Lisboa
           ('Viajante 2', 'MAD'),  # Madrid
           ('Viajante 3', 'CDG'),  # Paris
           ('Viajante 4', 'DUB'),  # Dublin
           ('Viajante 5', 'BRU'),  # Bruxelas
           ('Viajante 6', 'LHR')] # Londres 

In [81]:
pessoas[0]

('Viajante 1', 'LIS')

In [82]:
# Representação do aeroport do destino:
destino = 'FCO' # Roma

In [83]:
# Representação da solução: Será realizada com um VETOR de 12 POSIÇÕES
#   -> 6 voos de IDA + 6 voos de volta
# (ida e volta para cada pessoa)
# pessoa1: ida, volta
# pessoa2: ida, volta
#
# Lembrando que a ordem das pessoas é LISBOA, MADRID, PARIS, DUBLIN, BRUXELAS, LONRES
# Vetor resultante (exemplo) será algo assim:
# [0] -> DE LISBOA PARA ROMA, VOO 1 (SEGUNDO voo da lista de voos)
# [1] -> DE ROMA PARA LISBOA, VOO 2 (TERCEIRO voo da lista de voos)
# [2] -> DE MADRID PARA ROMA, VOO 3 (QUARTO voo da lista de voos)
# [3] -> DE ROMA PARA MADRID, VOO 2 (TERCEIRO voo da lista de voos)
# ...

# algo assim...:

agenda = [1,2, 3,2, 7,3, 6,3, 2,4, 5,3]
len(agenda)

12

In [84]:
len(agenda) // 2

6

In [85]:
#Importante lembrar para usarmos no algoritmo de otimização, a representação da origem
pessoas[1][1] #origem

'MAD'

## Função para impressão da agenda

In [86]:
from prettytable import PrettyTable #de https://pypi.org/project/prettytable/

def imprimir_voos(agenda):
  id_voo = -1
  preco_total = 0
  tabela = PrettyTable(['Pessoa', 'Origem', 'Destino', 'Hora de Saída', 'Hora de Chegada', 'Valor'])
  for i in range(len(agenda) // 2):
    nome = pessoas[i][0]   #nome da pessoa de de ida (usamos o nome da cidade)
    origem = pessoas[i][1] #sigla do aeroporto de origem 
    id_voo += 1 # auxiliar para percorrer a lista da agenda
    ida = voos[(origem, destino)][agenda[id_voo]] #indices da agenda nos voos
    preco_total += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    preco_total += volta[2]
    valor_pessoal = ida[2] + volta[2]


    #acrescenta a linha na tabela...:
    tabela.add_row([nome, origem, destino, ida[0], ida[1], ida[2]])
    tabela.add_row([nome, destino, origem, volta[0], volta[1], volta[2]])
    tabela.add_row([' ', ' ', ' ', ' ', 'Total deste passageiro:', valor_pessoal])
    tabela.add_row(['-', '-', '-', '-', '-', '-'])

  print(tabela)
  print(f'Valor total gasto: {preco_total}')

In [87]:
agenda = [2,3, 3,2, 7,3, 6,3, 2,4, 5,3]
imprimir_voos(agenda)

+------------+--------+---------+---------------+-------------------------+-------+
|   Pessoa   | Origem | Destino | Hora de Saída |     Hora de Chegada     | Valor |
+------------+--------+---------+---------------+-------------------------+-------+
| Viajante 1 |  LIS   |   FCO   |      9:15     |          12:03          |   99  |
| Viajante 1 |  FCO   |   LIS   |     11:07     |          13:24          |  171  |
|            |        |         |               | Total deste passageiro: |  270  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 2 |  MAD   |   FCO   |     11:01     |          12:39          |  260  |
| Viajante 2 |  FCO   |   MAD   |      9:11     |          10:42          |  172  |
|            |        |         |               | Total deste passageiro: |  432  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 3 |  CDG   |   FCO   |     17:07     |          20:04          | 

In [88]:
#apenas para ILUSTRAR a QUANTIDDE de COMBINAÇÕES possíveis para essa solução:
10**12

1000000000000

## Fitness function (Função de custo)


In [97]:
def fitness_function(agenda):
  id_voo = -1
  preco_total = 0
  for i in range(len(agenda) // 2):
    origem = pessoas[i][1] #coluna 1 (segunda) da lista de pessoas
    id_voo += 1
    ida = voos[(origem, destino)][agenda[id_voo]]
    preco_total += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    preco_total += volta[2]
  
  return preco_total

In [90]:
agenda = [1,2, 3,1, 4,1, 2,6, 8,3, 2,9]
fitness_function(agenda)

2741

In [91]:
!pip install mlrose #vamos utilizar essa bibliote pronta para as soluções inteligentes
#vide: https://mlrose.readthedocs.io/en/stable/

In [92]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

In [93]:
#para usarmos essa biblioteca, temos que informar qual a função de fitness, para isso podemos fazer assim:

fitness = mlrose.CustomFitness(fitness_function)

In [158]:
# Trabalharemos com números discretos (inteiros)
# parâmetros:
# - lenght: tamanho da solução
# - função de fitness
# - maximize = que tipo de solução -> True as maiores; False as menores
# - max_val: a quantidade de voos que temos em cada posição do vetor (o valor máximo de cada posição no vetor)

problema = mlrose.DiscreteOpt(length=12, 
                              fitness_fn=fitness, 
                              maximize=False, 
                              max_val=10)

## Hill Climb


In [ ]:
# o parâmetro random_state representa como é gerado o estado inicial
melhor_solucao, melhor_custo = mlrose.hill_climb(problema, random_state=1)

In [160]:
print(melhor_solucao)
print(melhor_custo)

[2 6 5 7 8 1 3 6 0 2 9 1]
1566.0


In [161]:
imprimir_voos(melhor_solucao)

+------------+--------+---------+---------------+-------------------------+-------+
|   Pessoa   | Origem | Destino | Hora de Saída |     Hora de Chegada     | Valor |
+------------+--------+---------+---------------+-------------------------+-------+
| Viajante 1 |  LIS   |   FCO   |      9:15     |          12:03          |   99  |
| Viajante 1 |  FCO   |   LIS   |     15:07     |          17:21          |  129  |
|            |        |         |               | Total deste passageiro: |  228  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 2 |  MAD   |   FCO   |     14:22     |          16:32          |  126  |
| Viajante 2 |  FCO   |   MAD   |     17:06     |          20:00          |   95  |
|            |        |         |               | Total deste passageiro: |  221  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 3 |  CDG   |   FCO   |     18:23     |          21:35          | 

In [162]:
voos['BRU', 'FCO']

[('6:12', '10:22', 230),
 ('7:53', '11:37', 433),
 ('9:08', '12:12', 364),
 ('10:30', '14:57', 290),
 ('12:19', '15:25', 342),
 ('13:54', '18:02', 294),
 ('15:44', '18:55', 382),
 ('16:52', '20:48', 448),
 ('18:26', '21:29', 464),
 ('20:07', '23:27', 473)]

In [163]:
voos['FCO', 'BRU']

[('6:09', '9:49', 414),
 ('7:57', '11:15', 347),
 ('9:49', '13:51', 229),
 ('10:51', '14:16', 256),
 ('12:20', '16:34', 500),
 ('14:20', '17:32', 332),
 ('15:49', '20:10', 497),
 ('17:14', '20:59', 277),
 ('18:44', '22:42', 351),
 ('19:57', '23:15', 512)]

## Simulated Annealing

In [164]:
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema, 
                                                          schedule=mlrose.decay.GeomDecay(init_temp=10000),
                                                          random_state=0)

In [165]:
print(melhor_solucao, melhor_custo)

[2 6 4 7 8 6 0 5 3 2 5 3] 1699.0


In [166]:
imprimir_voos(melhor_solucao)

+------------+--------+---------+---------------+-------------------------+-------+
|   Pessoa   | Origem | Destino | Hora de Saída |     Hora de Chegada     | Valor |
+------------+--------+---------+---------------+-------------------------+-------+
| Viajante 1 |  LIS   |   FCO   |      9:15     |          12:03          |   99  |
| Viajante 1 |  FCO   |   LIS   |     15:07     |          17:21          |  129  |
|            |        |         |               | Total deste passageiro: |  228  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 2 |  MAD   |   FCO   |     12:44     |          14:17          |  134  |
| Viajante 2 |  FCO   |   MAD   |     17:06     |          20:00          |   95  |
|            |        |         |               | Total deste passageiro: |  229  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 3 |  CDG   |   FCO   |     18:23     |          21:35          | 

## Algoritmo Genético (otimização)

In [184]:
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema,
                                                  pop_size=10000,
                                                  mutation_prob=0.2,
                                                  random_state=1)


In [181]:
melhor_custo, melhor_solucao

(1810.0, array([7, 7, 6, 5, 8, 4, 1, 6, 0, 2, 9, 1]))

In [185]:
imprimir_voos(melhor_solucao)

+------------+--------+---------+---------------+-------------------------+-------+
|   Pessoa   | Origem | Destino | Hora de Saída |     Hora de Chegada     | Valor |
+------------+--------+---------+---------------+-------------------------+-------+
| Viajante 1 |  LIS   |   FCO   |     16:51     |          19:09          |  147  |
| Viajante 1 |  FCO   |   LIS   |     16:35     |          18:56          |  144  |
|            |        |         |               | Total deste passageiro: |  291  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 2 |  MAD   |   FCO   |     15:58     |          18:40          |  173  |
| Viajante 2 |  FCO   |   MAD   |     14:19     |          17:09          |  190  |
|            |        |         |               | Total deste passageiro: |  363  |
|     -      |   -    |    -    |       -       |            -            |   -   |
| Viajante 3 |  CDG   |   FCO   |     18:23     |          21:35          | 